In [7]:
import torch
import numpy as np
import pandas as pd
import torch.optim as optim
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
import networkx as ne
from node2vec import Node2Vec
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [8]:
# 读loaded_data取保存的 CSV 文件
loaded_data = pd.read_csv('movie_score.csv')

# 显示加载的数据
#print(loaded_data)
loaded_data=loaded_data[loaded_data['Rate'] != 0]
print(loaded_data)

           User    Movie  Rate                       Time       Tag
1       1386692  1986338     3  2011-02-25T08:29:20+08:00   美国,2011
2       1386692  4268598     5  2011-02-14T10:05:42+08:00   日本,2011
3       1386692  1851857     4  2011-02-11T16:06:22+08:00   美国,2011
4       1386692  4023638     4  2011-02-10T19:25:12+08:00   英国,2011
5       1386692  1305903     3  2011-02-10T19:24:06+08:00  加拿大,2011
...         ...      ...   ...                        ...       ...
714934  1379646  1309004     5  2007-01-16T20:58:29+08:00       NaN
714967  1379646  1783772     5  2007-01-14T01:46:16+08:00       NaN
714984  1379646  1291859     5  2007-01-14T01:30:09+08:00       NaN
714993  1379646  1484091     5  2007-01-14T01:23:55+08:00       NaN
715018  1379646  1291836     4  2007-01-12T21:42:48+08:00       NaN

[523648 rows x 5 columns]


In [9]:
#TODO 处理加载的数据，得到item、user和star信息。
#合并相同的userids和itemsids
user_ids=loaded_data["User"].unique()
item_ids=loaded_data["Movie"].unique()
#创建user to row的字典
user_to_row ={user_id : idx for idx, user_id in enumerate(user_ids)}
item_to_row ={item_id : idx for idx, item_id in enumerate(item_ids)}

In [10]:
# 初始化一个空字典来存储社交关系
contact = {}

# 打开文件并读取内容
with open('Contacts.txt', 'r', encoding='utf-8') as f:
    for line in f:
        # 分割每一行的内容
        user, friends = line.strip().split(':')
        # 将朋友列表转换为整数列表
        if int(user) in user_to_row:
            friends_list = [user_to_row[int(friend)] for friend in friends.split(',') if int(friend) in user_to_row]
            # 将朋友列表添加到字典中
            contact[user_to_row[int(user)]] = friends_list

contact_sorted = {k: v for k, v in sorted(contact.items())}
# 打印字典的内容
print(contact_sorted)

{0: [190, 288, 972, 139, 286, 32, 85, 712], 1: [740, 383, 887, 66, 396], 2: [], 3: [788, 870, 793, 594, 109, 711, 993, 190, 854, 344, 638, 282, 139, 616, 392, 718, 275, 288, 6, 607, 155], 4: [967, 935, 768, 704, 415, 212, 82, 963, 25, 854, 194, 45, 803, 108, 60, 660, 906, 834, 257, 683, 474, 275, 1000, 699, 460, 879, 482, 50, 752, 706, 548, 867, 414, 104, 657, 455, 998, 161, 947, 607, 66, 918, 205, 410, 793, 769, 783, 593, 900, 968, 430, 125, 975, 794, 376, 549], 5: [855, 591], 6: [139, 895, 29, 10, 302, 614, 155, 1005, 892, 993, 921, 870, 476, 80, 578, 794, 465, 803, 1010, 193, 913, 137, 506, 221, 630, 486, 174, 618, 1002, 593, 323, 460, 832, 599, 854, 200, 1013, 209, 220, 37, 813, 275, 62], 7: [453, 241, 607, 897, 452, 63, 405, 388, 847, 577, 300, 750, 936, 341, 169, 740, 290, 176], 8: [854, 902, 70, 682, 241], 9: [201, 888, 522], 10: [420, 566, 854, 599, 921, 382, 720, 137, 1001, 1008, 563, 213, 11, 918, 207, 286, 398, 474, 702, 666, 275, 392, 139, 636, 612, 436, 425, 652, 968, 6, 4

In [11]:

import networkx as nx
G = nx.Graph()
for key, value in contact_sorted.items():
    for v in value:
        G.add_edge(key, v)
node2vec_model = Node2Vec(G, dimensions=200, walk_length=30, num_walks=200, workers=4,temp_folder="D:/temp")
model = node2vec_model.fit(window=10, min_count=1, batch_words=4)
social_embedding_temp = {node : model.wv[node] for node in G.nodes() if node in model.wv}

Computing transition probabilities:   0%|          | 0/1012 [00:00<?, ?it/s]



Generating walks (CPU: 1):   0%|          | 0/50 [00:00<?, ?it/s]


Generating walks (CPU: 1):   4%|▍         | 2/50 [00:01<00:35,  1.34it/s]


Generating walks (CPU: 1):   6%|▌         | 3/50 [00:03<00:55,  1.19s/it]


Generating walks (CPU: 1):   8%|▊         | 4/50 [00:05<01:02,  1.36s/it]


Generating walks (CPU: 1):  10%|█         | 5/50 [00:06<01:04,  1.43s/it]


Generating walks (CPU: 1):  12%|█▏        | 6/50 [00:08<01:08,  1.55s/it]


Generating walks (CPU: 1):  14%|█▍        | 7/50 [00:10<01:07,  1.57s/it]


Generating walks (CPU: 1):  16%|█▌        | 8/50 [00:11<01:08,  1.64s/it]


Generating walks (CPU: 1):  18%|█▊        | 9/50 [00:13<01:09,  1.69s/it]


Generating walks (CPU: 1):  20%|██        | 10/50 [00:15<01:06,  1.66s/it]

Generating walks (CPU: 1):  24%|██▍       | 12/50 [00:18<00:59,  1.58s/it]


Generating walks (CPU: 1):  26%|██▌       | 13/50 [00:19<00:57,  1.54s/it]


Generating walks (CPU: 1):  28%|██▊       | 14/50 [00:21<00:56,  1.57s/it]


Generating walk

In [12]:
social_embedding_dict = {}
for key, value in social_embedding_temp.items():
    social_embedding_dict[key] = value.tolist() 


In [13]:
import pickle
with open('social_embedding_dict_movie.pkl', 'wb') as f:
    pickle.dump(social_embedding_dict, f)

In [14]:
with open('social_embedding_dict_movie.pkl', 'rb') as f:
    social_embedding_dict = pickle.load(f)

In [15]:

class RatingDataset(Dataset):
    def __init__(self,data,user_to_row,item_to_row,social_embedding_dict):
        self.data=data
        #self.user_item_matrix=user_item_matrix
        self.user_to_row=user_to_row
        self.item_to_row=item_to_row
        self.social_embedding_dict = social_embedding_dict

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        row = self.data.iloc[idx]
        user = self.user_to_row[row['User']]
        movie = self.item_to_row[row['Movie']]
        rating = row['Rate'].astype('float32')
        social_embedding = self.social_embedding_dict.get(row['User'], np.zeros(200))
        return user, movie, rating,social_embedding
        
        
        

In [16]:
# embedding_dim为超参数，由用户定义
class MatrixFactorization(nn.Module):
    def __init__(self,num_users,num_movies,embedding_dim,hidden_state_social):
        super(MatrixFactorization,self).__init__()
        #词嵌入技术，将user和item分别嵌入为向量
        self.user_embeddings=nn.Embedding(num_users,embedding_dim)
        self.movie_embeddings=nn.Embedding(num_movies,embedding_dim)
        self.linear_embedding = nn.Linear(hidden_state_social,embedding_dim)
        self.output = nn.Linear(embedding_dim,6)

    def forward(self, user,movie,social_embedding):
        # 输出即为user矩阵和book矩阵相乘得到的结果
        user_embedding=self.user_embeddings(user)
        book_embedding=self.movie_embeddings(movie)
        social_embedding_proj = self.linear_embedding(social_embedding)
        user_intergrate = user_embedding + social_embedding_proj
        return (user_intergrate * book_embedding).sum(dim = 1)



In [17]:
#TODO 创建训练集和测试集的数据集对象和数据加载器
train_data, test_data = train_test_split(loaded_data, test_size=0.2, random_state=42)

train_dataset = RatingDataset(train_data, user_to_row, item_to_row,social_embedding_dict)
test_dataset = RatingDataset(test_data, user_to_row, item_to_row,social_embedding_dict)

train_dataloader = DataLoader(train_dataset, batch_size=4096, shuffle=True, drop_last = True)
test_dataloader = DataLoader(test_dataset, batch_size=4096, shuffle=False, drop_last = True)

embedding_dim=32

In [18]:
num_users=len(user_ids)
num_movies=len(item_ids)
print(num_users)
print(num_movies)

1014
1200


In [19]:
hidden_state_social = 200
model=MatrixFactorization(num_users,num_movies,embedding_dim,hidden_state_social).to(device)
criterion=nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
import numpy as np
model.train()
num_epochs = 20
lambda_b=0.001
lambda_u=0.001
for epoch in range(num_epochs):
    for user, movie, rating,social_embedding in train_dataloader:
        optimizer.zero_grad()
        user = user.to(device)
        movie = movie.to(device)
        rating = rating.to(device)
        social_embedding = social_embedding.float().to(device)
        output = model(user, movie,social_embedding)
        loss = criterion(output, rating) + lambda_u * model.user_embeddings.weight.norm(2) + lambda_b * model.movie_embeddings.weight.norm(2)
        loss.backward()
        optimizer.step()
    # 监控损失或其他性能指标
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")



Epoch 1/20, Loss: 15.377205848693848
Epoch 2/20, Loss: 5.971685409545898
Epoch 3/20, Loss: 3.2231497764587402
Epoch 4/20, Loss: 2.0456745624542236
Epoch 5/20, Loss: 1.498182773590088
Epoch 6/20, Loss: 1.135393500328064
Epoch 7/20, Loss: 0.9054871797561646
Epoch 8/20, Loss: 0.8253559470176697
Epoch 9/20, Loss: 0.7742931246757507
Epoch 10/20, Loss: 0.7271809577941895
Epoch 11/20, Loss: 0.6834542751312256
Epoch 12/20, Loss: 0.6609100103378296
Epoch 13/20, Loss: 0.6634713411331177
Epoch 14/20, Loss: 0.6435684561729431
Epoch 15/20, Loss: 0.6308813095092773
Epoch 16/20, Loss: 0.6345430016517639
Epoch 17/20, Loss: 0.6270876526832581
Epoch 18/20, Loss: 0.6028825044631958
Epoch 19/20, Loss: 0.6108394861221313
Epoch 20/20, Loss: 0.6113165616989136


In [20]:

model.eval()  # 将模型设置为评估模式，不进行梯度更新
test_loss = 0.0
with torch.no_grad():  # 不计算梯度
    for user, movie, rating,social_embedding in test_dataloader:
        user = user.to(device)
        movie = movie.to(device)
        rating = rating.to(device)
        social_embedding = social_embedding.float().to(device)
        output = model(user, movie,social_embedding)
        loss = criterion(output, rating)
        #print(f"Test Loss: {loss}")
        test_loss += loss.item()

# 计算测试性能指标，例如均方误差或其他指标
average_test_loss = test_loss / len(test_dataloader)
print(f"Average Test Loss: {average_test_loss}")

Average Test Loss: 0.5601732873916626


In [21]:
from sklearn.metrics import ndcg_score
#使用余弦相似度计算预测顺序和实际数据的差距
model.eval()  # 将模型设置为评估模式，不进行梯度更新
num_users=int(num_users)
num_movies=int(num_movies)
real_score_array=np.zeros((num_users,num_movies))
pred_score_array=np.zeros((num_users,num_movies))

with torch.no_grad():  # 不计算梯度
    for user, movie, rating,social_embedding in test_dataloader:
        user = user.to(device)
        movie = movie.to(device)
        social_embedding = social_embedding.float().to(device)
        rating = rating.to(device)
        pred = model(user, movie,social_embedding)
        for i in range(4096):
            real_score_array[int(user[i]),int(movie[i])]=int(rating[i])
            pred_score_array[int(user[i]),int(movie[i])]=float(pred[i])
cos_sum=0
ndcg_sum=0
#抽取前100组user
for i in range(100):
    #cal ndcg
    ndcg=ndcg_score(real_score_array[i].reshape(1,-1),pred_score_array[i].reshape(1,-1))
    ndcg_sum=ndcg_sum+ndcg
    vec1=np.argsort(real_score_array[i])
    vec2=np.argsort(pred_score_array[i])
    cos_sim = vec1.dot(vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    cos_sum=cos_sum+cos_sim
print(f"cos similarity={cos_sum/100}\nNDCG_score={ndcg_sum/100}")


        

cos similarity=0.977004532469569
NDCG_score=0.9624499048730432


In [22]:
from sklearn.metrics import ndcg_score
#print(real_score_array[0])
ndcg=ndcg_score(real_score_array[0].reshape(1,-1),pred_score_array[0].reshape(1,-1))
print(ndcg)

0.987882792499082
